<a href="https://colab.research.google.com/github/SujanKarna/RAG_ChatModel/blob/master/Langchain_chatModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install chromadb sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings

In [ ]:
# Initiaize the ChromaDb Client
client = chromadb.Client(Settings(persist_directory="./rag_store"))

In [ ]:
# Create a collection
collection = client.get_or_create_collection(name="rag_docs")

In [ ]:
# Prepare Documents
documents = [
    "Retrieval-Augmented Generation (RAG) combines search and generation.",
    "FAISS is a fast similarity search library developed by Facebook.",
    "ChromaDB supports metadata filtering and persistent storage.",
    "SentenceTransformers can convert text into semantic embeddings."
]

In [ ]:
# Embedd Documents
embedder = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = embedder.encode(documents).tolist()

In [ ]:
# Add Documents to ChromaDB
collection.add(
    documents=documents,
    embeddings=doc_embeddings,
    ids=[f"doc_{i}" for i in range(len(documents))]
)

In [ ]:
# Embedd the query
query = "What is RAG and how does it work?"
query_embedding = embedder.encode([query]).tolist()[0]

In [ ]:
# Retrieve top-K relevant documents
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3
)
retrieved_docs = results['documents'][0]

In [ ]:
# Check the retrieved documents
retrieved_docs

In [ ]:
# Assemble Context
context = "\n".join(retrieved_docs)
print(context)

In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import torch
from huggingface_hub import login

In [ ]:
# Generate answer with llm model
# Load Mistral model for generation

login(token="")
gen_model_name = "mistralai/Mistral-7B-Instruct-v0.2"
gen_tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
gen_model = AutoModelForCausalLM.from_pretrained(gen_model_name, torch_dtype=torch.float16, device_map="auto")

# Format prompt
prompt = f"""### Instruction:
Use the following context to answer the question.

### Context:
{context}

### Question:
{query}

### Answer:"""

# Generate response
inputs = gen_tokenizer(prompt, return_tensors="pt").to(gen_model.device)
with torch.no_grad():
    outputs = gen_model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )

answer = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)
print("🔍 Answer:\n", answer.split("### Answer:")[-1].strip())